<a href="https://colab.research.google.com/github/timosachsenberg/EuBIC2025/blob/main/notebooks/EUBIC_Task1_Peaks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install dependencies (for Google Colab)
!pip install -q pyopenms>=3.5.0 pyopenms-viz>=1.0.0

# Notebook 1 – From Proteins to Spectra: Digestion & First Look at LC–MS Data

In this notebook we will:

1. **Get acquainted with `pyopenms` and enzymatic digestion**  
   - Load a small FASTA file with a few protein sequences  
   - Perform an in-silico **Trypsin** digestion  
   - Generate realistic bottom-up **peptides** with length and missed-cleavage constraints  

2. **Take a first look at typical LC–MS data**  
   - Load an mzML file with **MS1 spectra**  
   - Visualize the data as a 2D **peak map (heatmap)**  
   - Extract and plot a **single MS1 spectrum**  

3. **Zoom into isotope patterns**  
   - Find an **isotope pattern** of an analyte  
   - Estimate the **charge state** and **mass** of the analyte  

4. **Compute the Total Ion Current (TIC)**  
   - Calculate the **TIC chromatogram** as the sum of all ion intensities over time  
   - Interpret the axes and what the TIC tells us about the LC–MS run

In [ ]:
%matplotlib inline
import pyopenms as oms
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np
print("pyOpenMS version:", oms.__version__)

pyOpenMS version: 3.5.0


## 1. Enzymatic digestion with Trypsin (in-silico)

**Aim of this task**

We want to get familiar with `pyopenms` and simulate a simple **bottom-up proteomics** scenario:

- We start from a small **FASTA file** with a few protein sequences.
- We apply a **Trypsin** digestion:
  - Trypsin cleaves **after K (Lys) and R (Arg)**, except when followed by **P (Proline)**.
- We allow up to **2 missed cleavages**, because the enzyme is not perfect.
- We only keep peptides with length between **7 and 40 amino acids**, so that they are realistic for LC–MS/MS.

At the end, we will:
- Print the protein entries from the FASTA loop
- Generate in-silico peptides for each protein
- Count and print the **total number of peptides**.


In [ ]:
# Download an example FASTA file
!wget -O "two_proteins.fasta" https://raw.githubusercontent.com/OpenMS/OpenMS/refs/heads/develop/src/tests/topp/Sequest_FASTAFile_test.fasta

fasta = oms.FASTAFile()
entries = []
fasta.load("two_proteins.fasta", entries)

print(f"Loaded {len(entries)} FASTA entries\n")

for i, entry in enumerate(entries):
    print(f"Entry {i}")
    print("  ID:     ", entry.identifier)
    print("  Desc:   ", entry.description)
    print("  Length: ", len(entry.sequence), "aa")
    print("  Seq (first 60 aa):", entry.sequence[:60], "...\n")

--2025-12-12 15:39:44--  https://raw.githubusercontent.com/OpenMS/OpenMS/refs/heads/develop/src/tests/topp/Sequest_FASTAFile_test.fasta
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 541 [text/plain]
Saving to: ‘two_proteins.fasta’

two_proteins.fasta  100%[===================>]     541  --.-KB/s    in 0s      

2025-12-12 15:39:45 (40.0 MB/s) - ‘two_proteins.fasta’ saved [541/541]

Loaded 2 FASTA entries

Entry 0
  ID:      P68509|1433F_BOVIN
  Desc:    
  Length:  245 aa
  Seq (first 60 aa): GDREQLLQRARLAEQAERYDDMASAMKAVTELNEPLSNEDRNLLSVAYKNVVGARRSSWR ...

Entry 1
  ID:      Q9CQV8|1433B_MOUSE
  Desc:    
  Length:  245 aa
  Seq (first 60 aa): TMDKSELVQKAKLAEQAERYDDMAAAMKAVTEQGHELSNEERNLLSVAYKNVVGARRSSW ...



Typically we only observe peptides of a certain size (about 7–40 amino acids) reliably. Very small ones are ambiguous and unstable; very large ones don’t ionize or fragment well.

In practice, proteolytic enzymes do not cleave perfectly. These missed cleavages produce longer peptides that still contain internal cleavage sites. Explore how allowing missed cleavages increases the number of generated peptides.



```
# Change this missed cleavages setting to see influence of missed cleavages on total number of peptides
digestion.setMissedCleavages()
```




Accounting for both missed cleavages and length restrictions will generate realistic peptide sequences that we could in principle measure by mass spectrometry.

In [ ]:
# Create a ProteaseDigestion object and configure Trypsin
digestion = oms.ProteaseDigestion()
digestion.setEnzyme("Trypsin")       # use Trypsin
digestion.setMissedCleavages(2)     # here allow up to 2 missed cleavages

# min and max length of peptide allow
min_len = 7
max_len = 40

# iterate over entries in fasta
for entry in entries:
    protein_seq = oms.AASequence.fromString(entry.sequence) # convert to AASequence object
    peptides = [] # empty list to store digested peptides list
    digestion.digest(protein_seq, peptides, min_len, max_len) # do the digestion

    print("\nNumber of in-silico peptides of protein", entry.identifier, ":", len(peptides))

    # Show a few example peptides
    print("\nExample peptides:")
    for pep in peptides[:5]:
      print(f"    {pep.toString()} (length {pep.size()})")



Number of in-silico peptides of protein P68509|1433F_BOVIN : 67

Example peptides:
    LAEQAER (length 7)
    YDDMASAMK (length 9)
    AVTELNEPLSNEDR (length 14)
    NLLSVAYK (length 8)
    VISSIEQK (length 8)

Number of in-silico peptides of protein Q9CQV8|1433B_MOUSE : 66

Example peptides:
    LAEQAER (length 7)
    YDDMAAAMK (length 9)
    AVTEQGHELSNEER (length 14)
    NLLSVAYK (length 8)
    VISSIEQK (length 8)


## 2. First look at MS1 data: peak map and single spectrum

**Aim of this task**

We now want to look at **real mass spectrometry data**.

- We load an mzML file containing **MS1 spectra**.
- We visualize the data as a 2D **peak map**:
  - **x-axis**: retention time (RT, usually in seconds or minutes)
  - **y-axis**: mass-to-charge ratio (**m/z**)
  - **color**: signal **intensity** at that RT–m/z position

This helps us get a global overview of the LC–MS run.

Then:

- We extract **one single MS1 spectrum**.
- We plot this spectrum as **intensity vs. m/z**.
- This allows us to see how a single snapshot of the LC–MS run looks like.


In [ ]:
# initialize MSExperiment object for 1D peak data
exp = oms.MSExperiment()
# used this from CPM github: small_with_TIC.mzML
oms.MzMLFile().load("small_with_TIC.mzML", exp) # load mzML

RuntimeError: the file 'test.mzML' could not be found

In [ ]:
# checkout pyOpenMS tutorial https://pyopenms.readthedocs.io/en/latest/user_guide/ms_data.html
def plot_spectra_2D(exp, ms_level=1, marker_size=5):
    fig = plt.figure(figsize=(12,8))
    exp.updateRanges()
    # extract each spectra from experiment
    for spec in exp:
        if spec.getMSLevel() == ms_level: # just take MS1 spectras
            mz, intensity = spec.get_peaks() # get peaks: mz corrsponding intensity
            p = intensity.argsort()  # sort by intensity to plot highest on top
            rt = np.full([mz.shape[0]], spec.getRT(), float) #Each spectrum has a single RT, but many m/z points, you broadcast the RT to match the length of the m/z array.
            plt.scatter(
                rt,
                mz[p],
                c=intensity[p],
                cmap="afmhot_r",
                s=marker_size,
                norm=colors.LogNorm(
                    exp.getMinIntensity() + 1, exp.getMaxIntensity()
                ),
            )
    plt.clim(exp.getMinIntensity() + 1, exp.getMaxIntensity())
    plt.xlabel("retention time (s)")
    plt.ylabel("m/z")
    plt.colorbar()
    plt.show()  # slow for larger data sets

# plot the spectra 2D
plot_spectra_2D(exp)


## 2.1 Plotting a single MS1 spectrum

Now we zoom into a **single MS1 spectrum**.

- A spectrum is a set of **peaks** measured at one retention time:
  - Each peak has an m/z value and an intensity.
- We plot **intensity vs. m/z**.

Tasks:

- Choose an MS1 spectrum index.
- Extract its m/z and intensity arrays via `get_peaks()`.
- Plot the spectrum as a stem plot.
- Clearly label axes:
  - x-axis: m/z
  - y-axis: intensity

optional: plot the spectrum with pyopenms `plot_spectrum`

In [ ]:
# Take one mass spectrum from provided experiment data
MS1_spectrum = exp[0] # By modifying the number, you can access the other spectrums
print("check spectrum level: ", MS1_spectrum.getMSLevel())


In [ ]:
# extract the peaks in form of mzs,intensities
mzs, intensities = MS1_spectrum.get_peaks()

print(f"Retention time: {MS1_spectrum.getRT():.1f} seconds "
      f"({MS1_spectrum.getRT()/60:.2f} minutes)")
print(f"Number of peaks: {len(mzs)}")

plt.figure(figsize=(12, 5))
plt.stem(mzs, intensities)#, use_line_collection=True)
plt.xlabel("m/z")
plt.ylabel("Intensity")
plt.title("Single MS1 spectrum (intensity vs. m/z)")
plt.tight_layout()
plt.show()

# Explanation:
# Each vertical line (stem) is a peak corresponding to ions with a certain m/z.
# The height of the line is the intensity, proportional to how many ions of that m/z were detected in this scan.

In [ ]:
# plot it with pyopenms plot_spectrum
oms.plotting.plot_spectrum(MS1_spectrum)

## 3. Isotope patterns: estimate charge and mass

In high-resolution MS1 spectra, analytes often appear as **isotope patterns**:

- Several peaks with the **same charge** but slightly different m/z values.
- The spacing between peaks is approximately **1 / z** m/z units, where *z* is the charge. Corresponding to an additional neutron.

We will:

1. **Zoom into a small m/z region** of one MS1 spectrum to visually find an isotope pattern.  
2. Visually measure the spacing between isotopic peaks to estimate the **charge z**.  
3. Estimate the **neutral mass** of the analyte:

m_neutral ≈ (m/z) * z - z * m_p


where \( m_p \) is the proton mass (~1.0073 Da).

In [ ]:
# Select again the same MS1 spectrum
#mzs, intensities = MS1_spectrum.get_peaks()

# Define an m/z window to zoom into.
# In a real training, participants could adjust these values interactively.
#mz_min_zoom = 645
#mz_max_zoom = 647

#mask = (mzs >= mz_min_zoom) & (mzs <= mz_max_zoom)
#mzs_zoom = mzs[mask]
#int_zoom = intensities[mask]

#plt.figure(figsize=(10, 4))
#plt.stem(mzs_zoom, int_zoom)#, use_line_collection=True)
#plt.xlabel("m/z")
#plt.ylabel("Intensity")
#plt.title(f"Zoomed MS1 spectrum: {mz_min_zoom}–{mz_max_zoom} m/z")
#plt.tight_layout()
#plt.show()

#print("Inspect the zoomed region to visually identify an isotope pattern.")
#print("Look for a group of peaks with nearly regular spacing.")


In [ ]:
# with pyopenms
ax = plt.subplot(1,1,1)
oms.plotting.plot_spectrum(MS1_spectrum,ax = ax)
# give the m/z range as xlim
ax.set_xlim([645,647]) # adjust these values interactively to find isotope pattern
plt.show()

We can see a clear isotope pattern. The peak at ~ 645.83 m/z is the lightest peak in the pattern and likely the monoisotopic peak. If we assume the peak at ~645.83 is indeed the peak of the monoisotopic molecule. The next peak (to the right) at ~646.33 corresponds to the molecule with one additional neutron in one of its atoms. We don't know which atom in each molecule has the additional neutron. It could be any. This means that the many ions that were recorded to form this isotopic peak, are likely a mixture of many isotopic variants of the molecule - each one with an additional neutron. The peak between the isotopic peaks are ~0.5 m/z. We know that a neutron adds ~1.0 Da (or u). If we observe a difference of 0.5 it tells us that the charge z of the ion is two.

## 4. Total Ion Current (TIC) chromatogram

Finally, we look at the **Total Ion Current chromatogram (TIC)**.

- The TIC shows the **total amount of ions** detected over time.
- More precisely, for each MS spectrum we sum **all peak intensities** and plot this sum vs. **retention time**.
- It is sometimes stored in the spectra files, but we will calculate it manually.

You can think of the TIC as a kind of **retention-time histogram** of the 2D peak map:

- We collapse the m/z axis by summing over all peaks.
- We keep only the retention time and the summed intensity.

We will:

1. Compute the TIC manually from the MS1 spectra.  
2. Plot TIC vs RT.  
3. Interpret the peaks in the TIC (e.g. they correspond to eluting analytes).

Think about how the TIC is related to the eluting analytes from the chromatography?


In [ ]:

tic_rts = []
tic_intensities = []

for spec in exp:
  if spec.getMSLevel() == 1:
      rt = spec.getRT()
      mzs, ints = spec.get_peaks()

      total_intensity = np.sum(ints)  # sum of all intensities in this spectrum

      tic_rts.append(rt)
      tic_intensities.append(total_intensity)

tic_rts = np.array(tic_rts)
tic_intensities = np.array(tic_intensities)

plt.figure(figsize=(10, 4))
plt.plot(tic_rts / 60.0, tic_intensities)
plt.xlabel("Retention time (minutes)")
plt.ylabel("Total ion current (sum of intensities)")
plt.title("Total Ion Current (TIC) chromatogram")
plt.tight_layout()
plt.show()


The TIC peaks indicate time regions where many ions (and thus analytes) elute.
Broad peaks can correspond to complex mixtures, sharp peaks to single analytes.

That's it for the first notebook.
You learned about:
*   Trypsin digestion with realistic constraints
*   Loading & visualizing MS data (peak + single spectrum)
*   Isotope pattern exploration (charge + neutral mass)
*   TIC computation with clear explanations of axes and colors